# Gradnja AI agentov s trajnim spominom z uporabo Cognee

Ta zvezek prikazuje, kako zgraditi inteligentne AI agente z naprednimi zmogljivostmi spomina z uporabo [**cognee**](https://www.cognee.ai/) - odprtokodnega AI spomina, ki združuje grafične baze znanja, semantično iskanje in upravljanje sej za ustvarjanje kontekstno zavednih AI sistemov.

## 🎯 Cilji učenja

Do konca tega vodiča boste razumeli, kako:
- **Zgraditi grafične baze znanja podprte z vdelavami**: Pretvoriti nestrukturirano besedilo v strukturirano, poizvedljivo znanje
- **Implementirati spomin sej**: Ustvariti večkratne pogovore z avtomatskim ohranjanjem konteksta
- **Shranjevati pogovore**: Po potrebi shraniti pomembne interakcije v dolgoročni spomin za prihodnjo uporabo
- **Poizvedovati z uporabo naravnega jezika**: Dostopati do zgodovinskega konteksta in ga uporabiti v novih pogovorih
- **Vizualizirati spomin**: Raziskovati odnose v grafični bazi znanja vašega agenta


## 🏗️ Kaj boste ustvarili

V tem vodiču bomo ustvarili **Pomočnika za programiranje** s trajnim spominom, ki:

### 1. **Izgradnja baze znanja**
   - Vključuje informacije o profilu in strokovnem znanju razvijalca
   - Obdeluje principe in najboljše prakse programiranja v Pythonu
   - Shranjuje zgodovinske pogovore med razvijalci in AI pomočniki

### 2. **Pogovori z zavedanjem seje**
   - Ohranja kontekst med več vprašanji v isti seji
   - Samodejno shranjuje vsak par vprašanje/odgovor za učinkovito iskanje
   - Ponuja skladne in kontekstualne odgovore na podlagi zgodovine pogovora

### 3. **Dolgoročni spomin**
   - Shranjuje pomembne pogovore v dolgoročni spomin
   - Pridobiva ustrezne spomine iz baze znanja in preteklih sej za obveščanje novih interakcij
   - Gradi rastočo bazo znanja, ki se sčasoma izboljšuje

### 4. **Inteligentno pridobivanje spomina**
   - Uporablja semantično iskanje, ki temelji na grafih, za iskanje ustreznih informacij v celotni shranjeni bazi znanja
   - Filtrira iskanja po podskupinah podatkov (informacije o razvijalcu proti principom)
   - Združuje več virov podatkov za zagotavljanje celovitih odgovorov


## 📋 Predpogoji in nastavitev

### Sistemskie zahteve

Pred začetkom se prepričajte, da imate:

1. **Python okolje**
   - Python 3.9 ali novejši
   - Virtualno okolje (priporočeno)
   
2. **Redis predpomnilnik** (zahtevan za upravljanje sej)
   - Lokalni Redis: `docker run -d -p 6379:6379 redis`
   - Ali uporabite upravljano storitev Redis
   
3. **Dostop do LLM API**
   - OpenAI API ključ ali drugi ponudniki (glejte [dokumentacijo](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Konfiguracija baze podatkov**
   - Privzeto ni potrebna konfiguracija. Cognee uporablja baze podatkov na osnovi datotek (LanceDB in Kuzu)
   - Po želji lahko nastavite Azure AI Search kot vektorsko shrambo (glejte [dokumentacijo](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Konfiguracija okolja

Ustvarite datoteko `.env` v svojem projektni mapi z naslednjimi spremenljivkami:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Razumevanje arhitekture spomina Cognee

### Kako deluje Cognee

Cognee ponuja napreden sistem spomina, ki presega preprosto shranjevanje ključ-vrednost:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Ključne komponente:

1. **Graf znanja**: Shranjuje entitete, odnose in semantične povezave
2. **Vektorske vdelave**: Omogočajo semantično iskanje po vseh shranjenih informacijah
3. **Predpomnilnik seje**: Ohranja kontekst pogovora znotraj in med sejami
4. **NodeSets**: Organizirajo podatke v logične kategorije za ciljno pridobivanje

### Vrste spomina v tem vodiču:

- **Trajni spomin**: Dolgoročno shranjevanje v grafu znanja
- **Sejni spomin**: Začasni kontekst pogovora v predpomnilniku Redis
- **Semantični spomin**: Iskanje podobnosti na osnovi vektorjev po vseh podatkih


## 📦 Namestitev potrebnih paketov

Namestite Cognee z Redis podporo za upravljanje sej:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inicializirajte okolje in naložite knjižnice

Prepričajte se, da:
1. Redis deluje (npr. prek Dockerja: `docker run -d -p 6379:6379 redis`)
2. So okoljske spremenljivke nastavljene pred uvozom modulov za predpomnjenje
3. Po potrebi ponovno zaženite jedro in zaženite celice po vrsti

Naslednja celica bo:
1. Naložila okoljske spremenljivke iz `.env`
2. Konfigurirala Cognee z vašimi nastavitvami LLM
3. Omogočila predpomnjenje za upravljanje sej
4. Preverila, ali so vsi komponenti pravilno povezani


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfiguracija shranjevalnih imenikov

Cognee uporablja dva ločena imenika za svoje delovanje:
- **Koren podatkov**: Shrani uvožene dokumente in obdelane podatke
- **Koren sistema**: Vsebuje podatkovno bazo grafa znanja in sistemske metapodatke

Za ta vodič bomo ustvarili ločene imenike, kot sledi:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Ponastavi stanje pomnilnika

Preden začnemo graditi naš pomnilniški sistem, poskrbimo, da začnemo na novo.

> 💡 **Namig**: Ta korak lahko preskočite, če želite ohraniti obstoječe spomine iz prejšnjih zagonov, ko boste ta zvezek uporabili kasneje.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 1. del: Gradnja baze znanja

### Viri podatkov za našega asistenta za razvijalce

Za ustvarjanje celovite baze znanja bomo uporabili tri vrste podatkov:

1. **Profil razvijalca**: Osebna strokovna znanja in tehnično ozadje
2. **Najboljše prakse za Python**: Zen Pythona s praktičnimi smernicami
3. **Zgodovinski pogovori**: Pretekle Q&A seje med razvijalci in AI asistenti

Ta raznolika podatkovna zbirka omogoča našemu agentu:
- Razumevanje tehničnega konteksta uporabnika
- Uporabo najboljših praks pri priporočilih
- Učenje iz prejšnjih uspešnih interakcij


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Pretvorba podatkov v graf znanja

Zdaj bomo surovo besedilo pretvorili v strukturiran spomin. Ta proces:

1. **Dodaja podatke v NodeSets**: Organiziranje informacij v logične kategorije
   - `developer_data`: Profil razvijalca in pogovori
   - `principles_data`: Najboljše prakse in smernice za Python

2. **Izvaja Cognify Pipeline**: Izvleče entitete, odnose in ustvari vektorske predstavitve
   - Prepozna ključne koncepte
   - Ustvari semantične povezave med povezanimi informacijami
   - Generira vektorske predstavitve

To lahko traja nekaj trenutkov, saj LLM obdeluje besedilo in gradi strukturo grafa:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Vizualizirajte graf znanja

Raziskujmo strukturo našega grafa znanja. Vizualizacija prikazuje:
- **Vozlišča**: Entitete, pridobljene iz besedila (koncepti, tehnologije, osebe)
- **Povezave**: Odnosi in povezave med entitetami
- **Grozdovje**: Povezani koncepti, združeni glede na semantično podobnost

Odprite ustvarjeno HTML datoteko v svojem brskalniku za interaktivno raziskovanje grafa:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Obogatite spomin z Memify

Funkcija `memify()` analizira graf znanja in ustvarja inteligentna pravila o podatkih. Ta proces:
- Prepoznava vzorce in najboljše prakse
- Ustvarja izvedljive smernice na podlagi vsebine
- Vzpostavlja povezave med različnimi področji znanja

Ta pravila agentu pomagajo pri sprejemanju bolj informiranih odločitev pri odgovarjanju na vprašanja. Zajem druge vizualizacije vam omogoča primerjavo, kako se graf zgosti, ko je obogaten.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 2. del: Inteligentno iskanje informacij

### Demonstracija 1: Integracija znanja med dokumenti

Zdaj, ko je naš graf znanja zgrajen, preverimo, kako Cognee združuje informacije iz več virov za odgovore na kompleksna vprašanja.

Prva poizvedba prikazuje:
- **Semantično razumevanje**: Iskanje relevantnih konceptov, tudi če niso izrecno omenjeni
- **Križno sklicevanje**: Združevanje profila razvijalca z načeli Python-a
- **Kontekstualno sklepanje**: Uporaba najboljših praks za specifične implementacije

### Demonstracija 2: Filtrirano iskanje z NodeSets

Druga poizvedba prikazuje, kako ciljati specifične podsklope grafa znanja:
- Uporablja parameter `node_name` za iskanje samo znotraj `principles_data`
- Ponuja osredotočene odgovore iz specifičnega področja znanja
- Koristno, kadar potrebujete informacije iz določenega področja


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 3. del: Nastavitev upravljanja sej

### Omogočanje pomnjenja pogovorov

Upravljanje sej je ključno za ohranjanje konteksta skozi več interakcij. Tukaj bomo:

1. **Inicializirali uporabniški kontekst**: Ustvarili ali pridobili uporabniški profil za sledenje sej
2. **Konfigurirali predpomnilniški mehanizem**: Povezali se z Redis za shranjevanje zgodovine pogovorov
3. **Omogočili spremenljivke seje**: Nastavili kontekstne spremenljivke, ki se ohranijo med poizvedbami

> ⚠️ **Pomembno**: To zahteva, da Redis deluje in da je `CACHING=true` nastavljeno v vašem okolju


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Pomožna funkcija: Ogled zgodovine seje

Ta pripomoček nam omogoča pregled zgodovine pogovorov, shranjenih v Redis. Uporaben je za:
- Odpravljanje napak pri upravljanju sej
- Preverjanje, ali se pogovori shranjujejo v predpomnilnik
- Razumevanje, kateri kontekst je na voljo agentu


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Seja 1: Laboratorij za podporo asinhronosti — Prvo vprašanje

Začnite sejo `async-support-lab` z vprašanjem o vzorcih asyncio, ki so prijazni do telemetrije, za obsežen spletni strgalnik. Graf že pozna asyncio, aiohttp in prakse spremljanja, zato naj odgovor odraža prejšnje pogovore, hkrati pa prilagodi odgovor na novo vprašanje.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Pregled pomnilnika seje 1 po prvem pogovoru

Zagon `show_history(session_1)` takoj po začetnem vprašanju potrjuje, da je Cognee shranil tako poziv kot odgovor v Redis. Videti bi morali en vnos z navodili za sočasnost.


In [ ]:
await show_history(session_1)

## Seja 1: Nadaljevanje o podatkovnih modelih

Naslednje vprašanje je: "Kdaj naj izberem dataclasses namesto Pydantic?" z uporabo istega ID-ja seje. Cognee naj združi Python načela in prejšnje pogovore o FastAPI, da poda podrobne nasvete—s čimer pokaže, da se kontekst prenese znotraj poimenovane seje.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Potrdite, da zgodovina seje 1 vsebuje oba obrata

Še en klic `show_history(session_1)` bi moral razkriti dva vnosa vprašanj in odgovorov. To ustreza koraku "ponovnega predvajanja spomina" v laboratoriju Mem0 in dokazuje, da dodatni obrati podaljšajo isti prepis.


In [ ]:
await show_history(session_1)

## Seja 2: Nit pregleda oblikovanja — Nova seja

Za prikaz izolacije med nitmi zaženemo `design-review-session` in prosimo za smernice beleženja za preglede incidentov. Čeprav je osnovna baza znanja enaka, nova ID seje ohranja prepise ločene.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Pregled seje 2 zgodovina

`show_history(session_2)` naj bi prikazal samo par poziv/odgovor za pregled zasnove. Primerjajte ga s sejo 1, da poudarite, kako Cognee ohranja neodvisne prepise, medtem ko ponovno uporablja skupni graf znanja.


In [ ]:
await show_history(session_2)

## Povzetek

Čestitke! Pravkar ste svojemu programerskemu asistentu dodali pravo dolgoročno plast spomina, ki jo poganja Cognee.

V tem vodiču ste vzeli surove vsebine za razvijalce (kodo, dokumentacijo, pogovore) in jih spremenili v graf + vektorski spomin, ki ga vaš agent lahko išče, uporablja za razmišljanje in nenehno izboljšuje.

Kaj ste se naučili:

1. **Od surovega besedila do AI spomina**: Kako Cognee obdeluje nestrukturirane podatke in jih spremeni v inteligenten, iskalni spomin z uporabo kombinirane arhitekture vektorjev + grafa znanja.

2. **Obogatitev grafa z memify**: Kako preseči osnovno ustvarjanje grafa in uporabiti memify za dodajanje izpeljanih dejstev ter bogatejših povezav na obstoječi graf.

3. **Različne strategije iskanja**: Kako poizvedovati v spominu z različnimi vrstami iskanja (Q&A, ki upošteva graf, dopolnjevanje v slogu RAG, vpogledi, surovi odlomki, iskanje kode itd.), odvisno od potreb vašega agenta.

4. **Vizualno raziskovanje**: Kako pregledati in odpravljati napake v tem, kar je Cognee ustvaril, z uporabo vizualizacij grafa in Cognee UI, da dejansko vidite, kako je znanje strukturirano.

5. **Spomin, ki upošteva seje**: Kako združiti kontekst posamezne seje s trajnim semantičnim spominom, da si agenti lahko zapomnijo med izvajanji, ne da bi pri tem prišlo do uhajanja informacij med uporabniki.


## Ključne točke
1. Spomin kot graf znanja, podprt z vektorji

    - **Strukturirano razumevanje**: Cognee združuje shrambo vektorjev in grafov, tako da so vaši podatki iskalni po pomenu in povezani z odnosi. Cognee privzeto uporablja baze podatkov na osnovi datotek (LanceDB za vektorje, Kuzu za grafovno bazo podatkov).

    - **Pridobivanje zavedanja o odnosih**: Odgovori temeljijo ne samo na "podobnem besedilu", temveč tudi na tem, kako so entitete povezane.

    - **Živ spomin**: Plast spomina se razvija, raste in ostaja poizvedljiva kot en povezan graf.

2. Načini iskanja in sklepanja
    - **Hibridno pridobivanje**: Iskanje združuje podobnost vektorjev, strukturo grafa in sklepanje LLM, od iskanja surovih delov do vprašanj, ki upoštevajo graf.

    - **Prilagodite način nalogi**: Uporabite načine v slogu dopolnjevanja, ko želite odgovore v naravnem jeziku, in načine za dele/besedila/grafe, ko vaš agent potrebuje surov kontekst ali želi samostojno sklepati.

3. Personalizirani agenti, zavedni seje
    - **Kontekst seje + dolgoročni spomin**: Cognee ločuje kratkoročni kontekst "niti" od dolgoročnega spomina na ravni uporabnika ali organizacije.

## Praktične uporabe

1. **Vertikalni AI agenti**

    Uporabite vzorec iz tega zvezka za pogon domensko pametnih kopilotov, ki temeljijo na Cognee kot njihovem jedru za pridobivanje in sklepanje:

- **Kopiloti za razvijalce**: Pregled kode, analiza incidentov in pomoč pri arhitekturi, ki pregleduje kodo, API-je, oblikovalske dokumente in vstopnice kot enoten graf spomina.

- **Kopiloti za stranke**: Agenti za podporo ali uspeh, ki črpajo iz produktne dokumentacije, pogostih vprašanj, CRM zapiskov in preteklih vstopnic z grafovno zavednim pridobivanjem in citiranimi odgovori.

- **Notranji strokovni kopiloti**: Pomočniki za politiko, pravne zadeve ali varnost, ki sklepajo na podlagi medsebojno povezanih pravil, smernic in zgodovinskih odločitev namesto izoliranih PDF-jev.

    Cognee je izrecno postavljen kot trajen, natančen spomin za AI agente, ki zagotavlja živ graf znanja, ki se poveže za vašim agentom in nadomesti ad-hoc kombinacije shramb vektorjev in prilagojene grafovne kode.

2. **Združevanje podatkovnih silosov v en spomin**

    Isti pristop vam pomaga zgraditi enotno plast spomina čez razpršene vire:

- **Od silosov do enega grafa**: Uvozite strukturirane (npr. baze podatkov) in nestrukturirane podatke (npr. dokumente, klepete) v en sam graf, podprt z vektorji, namesto ločenih indeksov za vsak sistem.

- **Sklepanje čez vire s citati**: Izvajajte večstopenjsko sklepanje čez vse—"združite" dnevnike, metrike in dokumente prek grafa—ter še vedno vrnite utemeljene odgovore s poreklom.

- **Središča znanja**: Za domene, kot so bančništvo ali izobraževanje, se Cognee že uporablja za združevanje PDF-jev, notranjih sistemov in podatkov aplikacij v en graf znanja z vektorji, tako da agenti lahko odgovarjajo na vprašanja s točnim, citiranim kontekstom.

## Naslednji koraki

Izvedli ste osnovno zanko spomina. Tukaj so naravne razširitve, ki jih lahko preizkusite sami (glejte [dokumentacijo Cognee](https://docs.cognee.ai/) za podrobnosti):

1. **Eksperimentirajte s časovno zavednostjo**: Vključite časovno funkcijo za izvleček dogodkov in časovnih žigov iz besedila.

2. **Uvedite sklepanje, ki temelji na ontologiji**: Določite OWL ontologijo za svojo domeno. Uporabite podporo za ontologijo v Cognee, da so izvlečene entitete in odnosi utemeljeni v tej shemi, kar izboljša kakovost grafa in domensko specifične odgovore.

3. **Dodajte povratno zanko**: Dovolite Cognee, da prilagodi uteži robov grafa na podlagi povratnih informacij uporabnikov, tako da se pridobivanje sčasoma izboljša in ne ostane statično.

4. **Prilagodite za personalizacijo in vedenje seje**: Uporabite ID-je uporabnikov, najemnike in nabore podatkov, da vsakemu posamezniku ali ekipi omogočite lasten pogled na skupni spominski mehanizem.

5. **Razširite na bolj zapletene agente**: Povežite Cognee z ogrodji agentov za gradnjo večagentnih sistemov, ki vsi delijo isto plast spomina. *Microsoft Agent Framework x Cognee vtičnik prihaja kmalu.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje AI [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem maternem jeziku naj se šteje za avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne prevzemamo odgovornosti za morebitne nesporazume ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
